#### Demo3 CAST Project

In [8]:
from CAST import *
import scanpy as sc
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")
work_dir = '$demo_path' #### input the demo path

In [9]:
output_path = f'{work_dir}/demo3_CAST_Projection/demo_output'
os.makedirs(output_path,exist_ok=True)
graph_list = {'rep1':['STAR_rep1','RIBO_rep1'], # source_sample, target_sample
              'rep2':['STAR_rep2','RIBO_rep2']} # source_sample, target_sample

#### CAST Project to project the STARmap data to the RIBOmap data

In [13]:
## This dataset contains 4 samples (STAR_rep1, RIBO_rep1, STAR_rep2, RIBO_rep2) and integrated features ('X_pca_harmony')
sdata = sc.read_h5ad(f'{output_path}/../data/demo3.h5ad')
sdata = preprocess_fast(sdata,mode = 'default')
batch_key = 'sample'

The `CAST_PROJECT()` function is used to project the `source_sample` (query sample) to `target_sample` (reference sample).


In [ ]:
sdata_refs = {}
list_ts = {}
color_dict = {
        'TEPN': '#256b00',
        'INH': '#ee750a',
        'CHO_PEP': '#f280cf',
        'DE_MEN': '#f24f4b',
        'AC': '#e8e879',
        'OLG': '#a8e1eb',
        'VAS': '#395ba8',
        'CHOR_EPEN': '#697491',
        'PVM': '#8803fc',
        'MLG': '#23ccb8',
        'OPC': '#667872',
        'Other': '#ebebeb'
    }
for rep in graph_list.keys():
    print(f'Start the {rep} samples:')
    source_sample, target_sample = graph_list[rep]
    output_path_t = f'{output_path}/{source_sample}_to_{target_sample}'
    os.makedirs(output_path_t,exist_ok=True)
    sdata_refs[rep],list_ts[rep] = CAST_PROJECT(
        sdata_inte = sdata[np.isin(sdata.obs[batch_key],[source_sample, target_sample])], # the integrated dataset
        source_sample = source_sample, # the source sample name
        target_sample = target_sample, # the target sample name
        coords_source = np.array(sdata[np.isin(sdata.obs[batch_key],source_sample),:].obs.loc[:,['x','y']]), # the coordinates of the source sample
        coords_target = np.array(sdata[np.isin(sdata.obs[batch_key],target_sample),:].obs.loc[:,['x','y']]), # the coordinates of the target sample
        scaled_layer = 'log1p_norm_scaled', # the scaled layer name in `adata.layers`, which is used to be integrated
        batch_key = batch_key, # the column name of the samples in `obs`
        source_sample_ctype_col = 'cell_type', # the column name of the cell type in `obs`
        output_path = output_path_t, # the output path
        integration_strategy = None, # 'Harmony' or None (use existing integrated features)
        color_dict = color_dict # the color dict for the cell type
    )